In [85]:
import glob
import os
import re
import pandas as pd

In [95]:
#Verzeichnis anpassen

os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
print(os.getcwd())

C:\Users\Katharina\Desktop\Weiterbildung\Bootcamp\Bootcamp\Final_project\data


In [87]:
# CSV-Dateien im aktuellen Verzeichnis finden
csv_files = glob.glob("tournament_statistics_*.csv")

# Zwei leere Listen für die Gruppen
df_list_30 = []
df_list_mixed = []

for file in csv_files:
    df = pd.read_csv(file, sep='\t')
    
    # Turniernummer aus Dateinamen extrahieren
    match = re.search(r'tournament_statistics_(\d+)\.csv', os.path.basename(file))
    if match:
        df["TournamentNo"] = match.group(1)
    else:
        df["TournamentNo"] = None

    # Unterscheidung nach Typ
    if 'ItemType' in df.columns:
        types = df['ItemType'].astype(str).unique()
        if set(types) == {'30'}:
            df_list_30.append(df)
        elif '11' in types:
            df_list_mixed.append(df)
        else:
            print(f"Unerwartete Typen in {file}: {types}")
    else:
        print(f"'Type' column missing in {file}, skipping...")

# Zu einem DataFrame zusammenfügen
combined_30 = pd.concat(df_list_30, ignore_index=True)
combined_mixed = pd.concat(df_list_mixed, ignore_index=True)


In [88]:
Daten_all = pd.concat([combined_30, combined_mixed], ignore_index=True, sort=False)

es wird weitergearbeitet mit combined_mixed

In [89]:
BeachMatches = pd.read_csv('BeachMatchList_full.csv', sep=';')

In [90]:
#Zusammenführen von Spielstatistiken und Matches

Combined_Stat = Daten_all.merge(BeachMatches, left_on = 'NoMatch', right_on='@No', how= 'left')

In [ ]:
testtourn = pd.read_csv('fehlendeTuniere_Nummern.csv', sep=';')
testtourn['TournamentNo'] = testtourn['TournamentNo'].astype(str)
werte = Combined_Stat['TournamentNo']

testtourn['TournamentNo'].isin(werte)

In [108]:
tourn1 = pd.read_csv('df_clean.csv', sep=';')
tourn2 = pd.read_csv('df_clean2.csv', sep=';')
tourn3 = pd.read_csv('clean45.csv', sep=';')
tourn4 = pd.read_csv('clean5.csv', sep=';')
tourn5 = pd.read_csv('Tournaments7575_7576.csv', sep=';')

Tournaments = pd.concat([tourn1, tourn2, tourn3, tourn4, tourn5])
Tournaments['No'] = Tournaments['No'].astype(str)

In [62]:
Dat_comp = Combined_Stat.merge(Tournaments, left_on='TournamentNo', right_on='No', how='left')

In [63]:
Dat_comp.to_csv('Daten_komplett.csv',sep=';', index=False)

Daten vorbereiten auf Koordinaten
- Aufbereitung von CountryName, Name_y, DefaultCity, Title

In [64]:
#1. Aufteilen in City befüllt und City nicht befüllt
Dat_comp['DefaultCity'].value_counts()

CityNA = Dat_comp[Dat_comp['DefaultCity'].isna()]
City_Full = Dat_comp[~Dat_comp['DefaultCity'].isna()]

In [45]:
CityNA.to_csv('DF_ortfehlt.csv' ,sep=';', index=False)

In [53]:
#Bearbeitet wieder einlesen
City_NA_bearbeitet = pd.read_csv('DF_ortfehlt.csv', sep=';')

C:\Users\Katharina\AppData\Local\Temp\ipykernel_16076\2066107218.py:2: DtypeWarning: Columns (29,59,71,72,78,104,106,113) have mixed types. Specify dtype option on import or set low_memory=False.
  City_NA_bearbeitet = pd.read_csv('DF_ortfehlt.csv', sep=';')


test

In [ ]:
#vergleich cityNA und city_na_bearbeitet


anti_join = City_NA_bearbeitet[~City_NA_bearbeitet['Version_y'].isin(CityNA['Version_y'])]
print(anti_join)
#alle in keineNAs enthalten

bearbeiten von City_NA .- City ist leer, Ort aus Name_y extrahieren

In [73]:
# City_NA_bearbeitet['Name_y'].unique()
# test = City_NA_bearbeitet[City_NA_bearbeitet['Name_y'].isna()]

#diese zeilen aus tunierNo 7575,7576 entfernen

deletecols = ['7575','7576']
City_NA_V2 = City_NA_bearbeitet[~City_NA_bearbeitet['TournamentNo'].astype(str).str.contains('|'.join(deletecols), case=False, na=False)]


In [ ]:
import pandas as pd
import numpy as np

# Angenommen, dein DataFrame heißt df und die Spalte mit Turniernamen heißt "TournamentName"
df = City_NA_V2#.dropna(subset=["Name_y"])  # NaN entfernen, falls vorhanden


# Schritt 1: Alle Einträge mit **nur einem Wort** (keine Leerzeichen)
df["Ort"] = np.nan  # Neue Spalte initialisieren

maske_ein_wort = df["Name_y"].str.strip().str.count(r'\s') == 0
df.loc[maske_ein_wort, "Ort"] = df.loc[maske_ein_wort, "Name_y"].str.strip()

# Schritt 2: Aus den übrigen Zeilen – Ort nach Bindestrich extrahieren
# Nur Zeilen ohne bereits gesetzten Ort
maske_mit_bindestrich = df["Ort"].isna() & df["Name_y"].str.contains(" - ")

# Regex: alles nach dem letzten " - " wird als Ort interpretiert
df.loc[maske_mit_bindestrich, "Ort"] = df.loc[maske_mit_bindestrich, "Name_y"].str.extract(r" - ([^-\n\r]+)$")[0].str.strip()

# Optional: Die Zeilen, die keinen Ort haben, explizit mit 'Unbekannt' befüllen (oder eine andere Default-Wert)
df["Ort"].fillna('Unbekannt', inplace=True)

# Optional: Duplikate in der Ort-Spalte beibehalten (keine Entfernung)
df_mit_ort = df[df["Ort"].notna()].copy()


In [ ]:
df_mit_ort[['Ort', 'Name_y']].drop_duplicates()

In [79]:
#ort zu Default_City
df_mit_ort['DefaultCity'] = df_mit_ort['Ort']
#df['DefaultCity'] = df['DefaultCity'].fillna(df['Ort'])
df_mit_ort = df_mit_ort.drop(columns=['Ort'])

Daten wieder zusammenfügen und dann bestimmte Zeilen entfernen

In [80]:
DF_comp = pd.concat([df_mit_ort, City_Full])

In [81]:
DF_comp.to_csv('Datenbestand1_3004.csv', index=False, sep=';')

In [84]:
test = DF_comp[DF_comp['DefaultCity']=='Unbekannt']
test[['DefaultCity', 'Name_y']].drop_duplicates()

,DefaultCity,Name_y
0,Unbekannt,Asian Women's Beach volleyball championship
208,Unbekannt,Asian Men's Beach volleyball championship
1024,Unbekannt,CEV Ljubljana Masters
1344,Unbekannt,CEV Alanya Masters
4044,Unbekannt,The Hague
4492,Unbekannt,CEV Pelhrimov Masters
4716,Unbekannt,2018 CEV European Championship
5004,Unbekannt,2018 CEV European Championship
5516,Unbekannt,Kish Island
6860,Unbekannt,Huntington Beach


die wörter unbekannt bearbeiten - mit Orten ersetzen